If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets as well as other dependencies. Uncomment the following cell and run it.

In [1]:
! pip install -U accelerate
! pip install -U transformers
import accelerate
import transformers

transformers.__version__, accelerate.__version__

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.8 MB/s eta 0:00:00


('4.35.2', '0.25.0')

In [2]:
! pip install datasets evaluate transformers rouge-score nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=b032311695542c9666b9bef611cd9cf0611e61292b459914f8105f212d396bf9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [3]:
import os
from huggingface_hub import login

#login(token=os.environ.get("HF_TOKEN"))
login(token="xxxxxxxxxxxxxxxxx",)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [5]:
from transformers.utils import send_example_telemetry

send_example_telemetry("summarization_notebook", framework="pytorch")

# Fine-tuning a model on a summarization task

In [6]:
model_checkpoint = "t5-small"

In [7]:
#mr_string = "[__DS_JOIN__ [__DG_NO__ [__ARG_TASK__ get_weather_attribute ] ] [__DG_INFORM__ [__ARG_TASK__ get_weather_attribute ] [__ARG_CONDITION_NOT__ thunderstorms ] [__ARG_LOCATION__ [__ARG_CITY__ St. Louis ] [__ARG_REGION__ Missouri ] ] [__ARG_DATE_TIME__ [__ARG_COLLOQUIAL__ today ] ] ] ] [__DG_INFORM__ [__ARG_TASK__ get_forecast ] [__ARG_TEMP_LOW__ 65 ] [__ARG_TEMP_HIGH__ 90 ] [__ARG_CLOUD_COVERAGE__ funnel cloud ] [__ARG_DATE_TIME__ [__ARG_COLLOQUIAL__ today ] ] [__ARG_LOCATION__ [__ARG_CITY__ St. Louis ] [__ARG_REGION__ Missouri ] ] ]"
mr_string = "[DG_NO ] [DG_INFORM [CONDITION light rain ] [HUMIDITY extremely humid ] [DATE_TIME [COLLOQUIAL today ] ] [LOCATION [CITY Palo Alto ] ] ] [DS_JUSTIFY [DG_RECOMMEND [ATTIRE_NOT jacket ] [LOCATION [CITY Palo Alto ] ] [DATE_TIME [COLLOQUIAL today ] ] ] [DG_INFORM [CONDITION_NOT cold ] [LOCATION [CITY Palo Alto ] ] [DATE_TIME [COLLOQUIAL today ] ] ] ]"

class MRTree:
  def __init__(self, item_type, item_value, item_children=None):
      self.item_type = item_type
      self.item_value = item_value
      self.item_children = item_children if item_children else []

  def to_dict(self):
      return {
          "item_type": self.item_type,
          "item_value": self.item_value,
          "item_children": [child.to_dict() for child in self.item_children]
      }
  def __str__(self):
    if self.item_children == []:
      return f"[{self.item_type} {self.item_value}]"
    chil=''
    for child in self.item_children:
      chil=chil+ str(child)
    return f"[{self.item_type} {self.item_value}{chil}]"

  def delexicalize_mr(self):
    if self.item_children == []:
      return f"[{self.item_type}]"
    chil=''
    for child in self.item_children:
      chil=chil+child.delexicalize_mr()
    return f"[{self.item_type}{chil}]"

  def reencode_mr(self):
    item_type = self.item_type
    if item_type.startswith('DS_'):
      item_type = item_type.replace('DS_',' please ') + ' that '
    elif item_type.startswith('DG_'):
      item_type = item_type.replace('DG_',' please ')+ ' '
    elif item_type.startswith('ARG_'):
      item_type = item_type.replace('ARG_',' ')+ ' is '
    else :
      item_type = ' ' + item_type + ' is '

    item_type = item_type.replace('_',' ')
    item_type = item_type.replace('  ',' ')
    item_type = item_type.lower()
    returnval=''
    if self.item_children == []:
      returnval= f" {item_type} {self.item_value} "
    else:
      chil=''
      for child in self.item_children:
        chil=chil+child.reencode_mr()
      returnval =f" {item_type} {self.item_value} {chil} "

    returnval = returnval.replace('  ',' ')
    return returnval

def parse_mr_string(mr_string):
  import pyparsing as pp
  mr_string = mr_string.replace('  ', ' ')
  mr_string = mr_string.replace('__', '')
  mr_string_root="[ROOT"+mr_string+"]"
  nested_expression_parser = pp.nestedExpr("[", "]",content=pp.CharsNotIn("[]"))
  nested_list = nested_expression_parser.parseString(mr_string_root).asList()[0][1:]
  return nested_list

def parse_mr_list(lst):
  mr_list=[]
  if lst == []:
    return None
  for item in lst:
    item_type=item[0].strip().split(' ')[0]
    item_value=item[0].strip().split(' ')[1:]
    item_value=' '.join(item_value)
    item_children=[]
    last=item[-1]
    if type(last)==str:
      item_children=item[1:-1]
    else:
      item_children=item[1:]
    item_children =parse_mr_list(item_children)
    mr_list.append(MRTree(item_type, item_value, item_children))
  return mr_list


def parse_mr(mr_string):
  return parse_mr_list(parse_mr_string(mr_string))



In [8]:
def serialize_mr(mr_list):
  import json
  mr_json=[]
  for item in mr_list:
      mr_json.append(item.to_dict())
  return mr_json

def stringify_mr(mr_list):
  mr_string=""
  for item in mr_list:
    mr_string = mr_string+ str(item)
  return mr_string

def humanize_mr(mr_list):
  readable_mr=''
  for item in mr_list:
    readable_mr = readable_mr+ item.reencode_mr()
  return readable_mr


def delexicalize_mr(mr_list):
  mr_delexicalize=''
  for item in mr_list:
    mr_delexicalize = mr_delexicalize+ item.delexicalize_mr()
  return mr_delexicalize


In [9]:
from datasets import Dataset

def filter_unique_mrs(mr_dataset):

    """
    Filter the list of MRs to retain only unique ones based on their delexicalized structures
    and return a HuggingFace Dataset.
    """
    unique_mrs_data = []
    seen_structures = set()

    for mr in mr_dataset:
        # Delexicalize the MR
        print(1)
        print(mr)
        delex_mr = delexicalize_mr(parse_mr(mr["tree_str_mr"]))

       # Check if this structure has been seen before
        if delex_mr not in seen_structures:
            seen_structures.add(delex_mr)
            unique_mrs_data.append(mr)  # Add the original MR to the unique list

    # Convert list of dictionaries to a dictionary of lists
    unique_mrs_dict = {key: [d[key] for d in unique_mrs_data] for key in unique_mrs_data[0]}

    # Convert the dictionary of lists to a HuggingFace Dataset
    unique_mrs_dataset = Dataset.from_dict(unique_mrs_dict)
    return unique_mrs_dataset

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [10]:
#!pip install datasets

In [11]:
from datasets import DatasetDict
from datasets import load_dataset
from evaluate import load
 raw_datasets= load_dataset('GEM/conversational_weather', split="train+test+validation")
#aw_datasets = load_dataset('GEM/conversational_weather',split=['train[10:20]', 'test'])

 #raw_datasets2 = load_dataset("xsum")
metric = load("rouge")

In [12]:
#raw_datasets

In [13]:
unique_raw_datasets= filter_unique_mrs(raw_datasets)

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
#unique_raw_datasets.save_to_disk('/content/drive/MyDrive/Colab-Notebooks/INF503_SEP2023_A2_V2/Dataset/')

In [18]:
from datasets import load_from_disk
unique_raw_datasets =load_from_disk('/content/drive/MyDrive/Colab-Notebooks/INF503_SEP2023_A2_V2/unique_raw_datasets/')

In [ ]:
#raw_datasets["train"]=Dataset.from_dict(filter_unique_mrs(raw_datasets["train"]))
#raw_datasets["validation"]=Dataset.from_dict(filter_unique_mrs(raw_datasets["validation"]))
#raw_datasets["test"]=Dataset.from_dict(filter_unique_mrs(raw_datasets["test"]))

In [20]:
unique_raw_datasets

Dataset({
    features: ['gem_id', 'data_id', 'user_query', 'tree_str_mr', 'response', 'target', 'references'],
    num_rows: 5367
})

In [19]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [21]:
#show_random_elements(raw_datasets["train"])
show_random_elements(unique_raw_datasets)

,gem_id,data_id,user_query,tree_str_mr,response,target,references
0,default-train-5231,1140296,What time will it be the hottest today?,[__DS_CONTRAST__ [__DG_ERROR__ [__ARG_TASK__ get_weather_attribute ] [__ARG_ERROR_REASON__ unsupported_question ] ] [__DG_INFORM__ [__ARG_TASK__ get_forecast ] [__ARG_TEMP_LOW__ 42 ] [__ARG_TEMP_HIGH__ 60 ] [__ARG_TEMP_UNIT__ fahrenheit ] ] ],"[__DS_CONTRAST__ [__DG_ERROR__ I can't answer that, ] but [__DG_INFORM__ the high will be [__ARG_TEMP_HIGH__ __ARG_TEMP_HIGH__ ] [__ARG_TEMP_UNIT__ __ARG_TEMP_UNIT__ ] with a low of [__ARG_TEMP_LOW__ __ARG_TEMP_LOW__ ] ] ]","[__DS_CONTRAST__ [__DG_ERROR__ I can't answer that, ] but [__DG_INFORM__ the high will be [__ARG_TEMP_HIGH__ __ARG_TEMP_HIGH__ ] [__ARG_TEMP_UNIT__ __ARG_TEMP_UNIT__ ] with a low of [__ARG_TEMP_LOW__ __ARG_TEMP_LOW__ ] ] ]","[[__DS_CONTRAST__ [__DG_ERROR__ I can't answer that, ] but [__DG_INFORM__ the high will be [__ARG_TEMP_HIGH__ __ARG_TEMP_HIGH__ ] [__ARG_TEMP_UNIT__ __ARG_TEMP_UNIT__ ] with a low of [__ARG_TEMP_LOW__ __ARG_TEMP_LOW__ ] ] ]]"
1,default-train-1519,1126787,What's next week's forecasts for Tucson?,[__DS_CONTRAST__ [__DG_ERROR__ [__ARG_TASK__ get_forecast ] [__ARG_BAD_ARG__ date_time_range ] [__ARG_ERROR_REASON__ unsupported_question ] ] [__DS_JOIN__ [__DG_INFORM__ [__ARG_TASK__ get_forecast ] [__ARG_TEMP_HIGH_SUMMARY__ between 56 and 60 ] [__ARG_DATE_TIME_RANGE__ [__ARG_COLLOQUIAL__ rest of the week ] ] [__ARG_LOCATION__ [__ARG_CITY__ Tucson ] [__ARG_REGION__ Arizona ] ] ] [__DG_INFORM__ [__ARG_TASK__ get_forecast ] [__ARG_DATE_TIME_RANGE__ [__ARG_COLLOQUIAL__ rest of the week ] ] [__ARG_LOCATION__ [__ARG_CITY__ Tucson ] [__ARG_REGION__ Arizona ] ] [__ARG_TEMP_LOW_SUMMARY__ between 32 and 48 ] ] [__DS_CONTRAST__ [__DG_INFORM__ [__ARG_TASK__ get_forecast ] [__ARG_CONDITION__ heavy fog patches ] [__ARG_DATE_TIME__ [__ARG_WEEKDAY__ Monday ] ] [__ARG_LOCATION__ [__ARG_CITY__ Tucson ] [__ARG_REGION__ Arizona ] ] ] [__DG_INFORM__ [__ARG_TASK__ get_forecast ] [__ARG_DATE_TIME_RANGE__ [__ARG_COLLOQUIAL__ next week ] ] [__ARG_LOCATION__ [__ARG_CITY__ Tucson ] [__ARG_REGION__ Arizona ] ] ] ] ] ],"[__DS_CONTRAST__ [__DG_ERROR__ Sorry, I can't get the weather for that [__ARG_BAD_ARG__ __ARG_BAD_ARG__ ] ] , but [__DS_JOIN__ [__DG_INFORM__ expect highs [__ARG_TEMP_HIGH_SUMMARY__ between <number> and <number> ] ] and [__DG_INFORM__ lows [__ARG_TEMP_LOW_SUMMARY__ between <number> and <number> ] [__ARG_DATE_TIME_RANGE__ [__ARG_COLLOQUIAL__ next week ] ] [__ARG_LOCATION__ in [__ARG_CITY__ __ARG_CITY__ ] , [__ARG_REGION__ __ARG_REGION__ ] ] ] . [__DS_CONTRAST__ [__DG_INFORM__ There will be [__ARG_CONDITION__ heavy fog patches ] on [__ARG_DATE_TIME__ [__ARG_WEEKDAY__ __ARG_WEEKDAY__ ] ] ] , but [__DG_INFORM__ not the [__ARG_DATE_TIME_RANGE__ [__ARG_COLLOQUIAL__ rest of the week ] ] ] ] ] ]","[__DS_CONTRAST__ [__DG_ERROR__ Sorry, I can't get the weather for that [__ARG_BAD_ARG__ __ARG_BAD_ARG__ ] ] , but [__DS_JOIN__ [__DG_INFORM__ expect highs [__ARG_TEMP_HIGH_SUMMARY__ between <number> and <number> ] ] and [__DG_INFORM__ lows [__ARG_TEMP_LOW_SUMMARY__ between <number> and <number> ] [__ARG_DATE_TIME_RANGE__ [__ARG_COLLOQUIAL__ next week ] ] [__ARG_LOCATION__ in [__ARG_CITY__ __ARG_CITY__ ] , [__ARG_REGION__ __ARG_REGION__ ] ] ] . [__DS_CONTRAST__ [__DG_INFORM__ There will be [__ARG_CONDITION__ heavy fog patches ] on [__ARG_DATE_TIME__ [__ARG_WEEKDAY__ __ARG_WEEKDAY__ ] ] ] , but [__DG_INFORM__ not the [__ARG_DATE_TIME_RANGE__ [__ARG_COLLOQUIAL__ rest of the week ] ] ] ] ] ]","[[__DS_CONTRAST__ [__DG_ERROR__ Sorry, I can't get the weather for that [__ARG_BAD_ARG__ __ARG_BAD_ARG__ ] ] , but [__DS_JOIN__ [__DG_INFORM__ expect highs [__ARG_TEMP_HIGH_SUMMARY__ between <number> and <number> ] ] and [__DG_INFORM__ lows [__ARG_TEMP_LOW_SUMMARY__ between <number> and <number> ] [__ARG_DATE_TIME_RANGE__ [__ARG_COLLOQUIAL__ next week ] ] [__ARG_LOCATION__ in [__ARG_CITY__ __ARG_CITY__ ] , [__ARG_REGION__ __ARG_REGION__ ] ] ] . [__DS_CONTRAST__ [_

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [22]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

## Preprocessing the data

In [23]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [24]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "paraphrase: "
else:
    prefix = ""

In [41]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(raw_dataset):
    print(raw_dataset)
    # Tokenize the inputs and labels. Add the special tokens
    inputs = [doc["user_query"] + " [SEP] "+ humanize_mr(parse_mr(doc["tree_str_mr"])) for doc in raw_dataset]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    targets = list(map(lambda x:x[0], raw_dataset["references"]))
    # Setup the tokenizer for targets
    labels = tokenizer(text_target=targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [44]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(raw):
    print(raw)
    # Tokenize the inputs and labels. Add the special tokens
    input =  raw["user_query"] + " [SEP] "+ humanize_mr(parse_mr(raw["tree_str_mr"]))
    model_input = tokenizer(input, max_length=max_input_length, truncation=True)
    target = raw["references"][0]
    # Setup the tokenizer for targets
    label = tokenizer(text_target=target, max_length=max_target_length, truncation=True)

    model_input["labels"] = label["input_ids"]
    return model_input

df = pd.DataFrame(td)
for column, typ in td.features.items():
    if isinstance(typ, datasets.ClassLabel):
        df[column] = df[column].transform(lambda i: typ.names[i])
display(HTML(df.to_html()))

td = unique_raw_datasets.select(range(0,10))

inputs = [doc["user_query"] + " [SEP] "+ humanize_mr(parse_mr(doc["tree_str_mr"])) for doc in td]
tokenizer(inputs, max_length=max_input_length, truncation=True)

targelist = list(map(lambda x:x[0], td["references"]))
labels = tokenizer(text_target=targelist, max_length=max_target_length, truncation=True)
labels


In [26]:
train_dataset = unique_raw_datasets.select(range(0,250))
test_dataset = unique_raw_datasets.select(range(1000,1100))
validation_dataset = unique_raw_datasets.select(range(2000,2100))

In [74]:
test_recod = train_dataset[0]

input  =  test_recod["user_query"] + " [SEP] "+ humanize_mr(parse_mr(test_recod["tree_str_mr"]))
input

'Is it cold outside [SEP]  please yes  task is get_weather_attribute  please inform  task is get_forecast temp high summary is low teens date time is colloquial is today location is city is Newport  please inform  task is get_forecast date time is colloquial is today location is city is Newport temp low is -5 '

In [73]:
#train_dataset, validation_dataset= unique_raw_datasets.train_test_split(test_size=0.1).values()
#import datasets

#final_dataset = datasets.DatasetDict({"train":train_dataset,"test":test_dataset,"validation":validation_dataset})
tokenized_train_dataset =  train_dataset.map(preprocess_function2, batched=False)
tokenized_test_dataset =  test_dataset.map(preprocess_function2, batched=False)
tokenized_validation_dataset =  validation_dataset.map(preprocess_function2, batched=False)



This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
#tokenized_datasets = final_dataset.map(preprocess_function, batched=True)

In [53]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [62]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-weather",
    evaluation_strategy = "epoch",
    learning_rate=1e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=50,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/t5-finetuned-xsum"` or `"huggingface/t5-finetuned-xsum"`).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [63]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [64]:
import nltk
import numpy as np

  def compute_metrics(eval_pred):
      predictions, labels = eval_pred
      decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
      # Replace -100 in the labels as we can't decode them.
      labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
      decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

      # Rouge expects a newline after each sentence
      decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
      decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

      # Note that other metrics may not have a `use_aggregator` parameter
      # and thus will return a list, computing a metric for each sentence.
      result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
      # Extract a few results
      result = {key: value * 100 for key, value in result.items()}

      # Add mean generated length
      prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
      result["gen_len"] = np.mean(prediction_lens)

      return {k: round(v, 4) for k, v in result.items()}

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [65]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [66]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

We can now finetune our model by just calling the `train` method:

In [67]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.200626,13.497800,1.170700,10.325300,10.630900,19.000000
2,No log,3.173289,13.455700,1.170700,10.274100,10.585900,19.000000
3,No log,3.148754,13.501700,1.170700,10.324100,10.624800,19.000000
4,No log,3.121171,13.446000,1.131300,10.292000,10.578700,19.000000
5,No log,3.088663,13.552200,1.171300,10.235600,10.627200,19.000000
6,No log,3.056265,13.558400,1.171800,10.207900,10.621800,19.000000
7,No log,3.029503,13.452000,1.171800,10.254300,10.621800,19.000000
8,No log,3.004629,13.197400,1.171800,10.084200,10.440800,19.000000
9,No log,2.984097,13.221500,1.171800,10.134100,10.550300,19.000000
10,No log,2.964417,13.225400,1.241300,10.135400,10.568500,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=800, training_loss=3.355691146850586, metrics={'train_runtime': 254.9015, 'train_samples_per_second': 49.039, 'train_steps_per_second': 3.138, 'total_flos': 480845101400064.0, 'train_loss': 3.355691146850586, 'epoch': 50.0})

You can now upload the result of the training to the Hub, just execute this instruction:

In [68]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1701711071.2cbdf34ced0f.1595.3:   0%|          | 0.00/31.9k [00:00<?, ?B/s]

'https://huggingface.co/elrashid/t5-small-finetuned-weather/tree/main/'

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```

In [70]:
from transformers import AutoModelForSeq2SeqLM

model_pretrained= AutoModelForSeq2SeqLM.from_pretrained("elrashid/t5-small-finetuned-weather")

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [77]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
text = 'Is it cold outside [SEP]  please yes  task is get_weather_attribute  please inform  task is get_forecast temp high summary is low teens date time is colloquial is today location is city is Newport  please inform  task is get_forecast date time is colloquial is today location is city is Newport temp low is -5'
mname = 'elrashid/t5-small-finetuned-weather'
tokenizer = AutoTokenizer.from_pretrained(mname)
model = AutoModelForSeq2SeqLM.from_pretrained(mname)
input_ids = tokenizer.encode(text, return_tensors="pt")
outputs = model.generate(input_ids,max_new_tokens=10000 )
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded) #Nice to meet you

Is it cold outside [SEP] please yes task is get_weather_attribute please inform task is get_forecast temp high summary is low teens date time is colloquial is today location is city is Newport please inform task is get_forecast temp high summary is low teens date time is colloquial is today location is city is Newport please inform task is get_forecast temp high summary is low teens date time is colloquial is today location is city is Newport please inform task is get
